# Introduction

## Imports

In [1]:
import pandas as pd

import numpy as np

import os

In [2]:
# This will allow you to see all column names & rows when you are doing .head(). None of the column name will be truncated.
# source: https://stackoverflow.com/questions/49188960/how-to-show-all-of-columns-name-on-pandas-dataframe

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

## Data info

In [3]:
for file in os.listdir('../data/'):
    print(file + '\t\t\t\t\t\t' + str(os.stat("../data/" + file).st_size/1000000))

target_competencia_ids.csv						0.200915
zips						0.004096
clicks.csv						16.147446
auctions.csv						2412.11824
installs.csv						123.502317
events.csv						2252.988966


# Useful functions

## General

In [4]:
# days to consider
all_days = [18,19,20,21,22,23,24,25,26]

secs_in_3_days = 3*24*60*60

In [5]:
def get_n_3_days(n):
    """
    get nth block of 3 consecutive days
    n can go from 1 to 7.
    If n == 8, then last two days are given.
    If n == 9, then last day is given.
    """
    n -= 1
    return all_days[n:n+3]

### Target related

In [6]:
# load target
def load_target():
    target = pd.read_csv('../data/target_competencia_ids.csv')

    # to avoid misunderstandings with data when predicting, and avoid accidentally predicting value zero
#     target.obj = np.nan
    
    return target

# para que quede cargado desde el principio
target = load_target()

# target ids related
def get_target_ids():
    """ get all target ids """
    return target['ref_hash'].apply(lambda x: x[:-3]).unique()

def get_target_ids_chunk(chunk_num):
    """ chunk num can go from 1 to 41 """
    chunk_size = 100
    start = (chunk_num - 1) * chunk_size
    stop = chunk_size * chunk_num
    return get_target_ids()[start:stop]

### Predictions related

In [139]:
# para guardar predicciones
import time
def _get_filename(my_name, timestamp):
    return "../predictions/" + timestamp + " by " + my_name + ".csv"

def _save_description(authors_name, timestamp, submission_description):
    f = open("../predictions/" + authors_name + ".txt","a")
    f.write(timestamp + ": " + submission_description + '\n')
    f.close()

def save_submission(submission_df, authors_name="fabri", description = "no description"):
    timestamp = time.strftime("%Y.%m.%d - %H:%M:%S")
    submission_df.to_csv(_get_filename(authors_name, timestamp), index=False)
    _save_description(authors_name, timestamp, description)

In [8]:
def store_predictions(target_df, new_values, value_column_name, suffix):
    """
    adds predictions from value_column_name from new_values df
    to target_df merging by ref_hash and the given suffix 
    suffix: "_st" for auction prediction
            "_sc" for conversion prediction
    """
    new_values['ref_hash'] = new_values['ref_hash'] + suffix
    
    target_df = target_df.merge(new_values[['ref_hash',value_column_name]], how='left', on='ref_hash')
    
    target_df.fillna(0, inplace=True)
    
#     assign values to 'obj' column and remove the column added on merge.
#     after sum, fillna is needed because there are values which are left as NaNs.
    target_df['obj'] = target_df['obj'] + target_df[value_column_name]
    
    target_df.drop([value_column_name], axis='columns', inplace=True)
    return target_df

In [9]:
# play a sound
import os
def ring(duration = 1, freq = 1500):
    """ play tone of duration in seconds and freq in Hz. """
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

### ML related

In [10]:
def set_source_col(df, source):
    """
    for a given dataframe, create a column indicating
    from which csv file it originated
    """
    df['source_csv'] = source

    
def process_time_diffs(df):
    """
    create column indicating difference between two
    consecutive registers for each device id
    """
    asdf = df
    
    asdf = asdf.sort_values(['date'])
    
    asdf['diff'] = asdf.groupby(['ref_hash'])['date'].diff()

    asdf['diff'].fillna(value=asdf['date']-asdf['date'].dt.floor('d'), inplace=True)

    asdf['diff_in_sec'] = asdf['diff'].dt.total_seconds()

    asdf.drop('diff', axis='columns', inplace=True)
    
    return asdf


def process_time_diffs_vs_min_day(df):
    """
    create column indicating difference between
    time in registers and min day in df
    """
    asdf = df

    min_timestamp = asdf['date'].min().floor('d')

    asdf['diff'] = asdf['date'] - min_timestamp

    asdf['diff_in_sec'] = asdf['diff'].dt.total_seconds()

    asdf.drop('diff', axis='columns', inplace=True)
    
    return asdf

def is_source_that_defines_death(data, source_that_defines_death):
    return data == source_that_defines_death

def set_observed_column(df, csv_source_that_defines_death):
    """
    create column indicating if death has been observed or not.
    """
    asdf = df
    
    asdf['observed'] = asdf.source_csv.apply(lambda x: is_source_that_defines_death(x,csv_source_that_defines_death))
    
    return asdf


def fill_with_mode(x):
    """
    If there is any value present in group, fill nans with the mode of the group. 
    If there are all nans, leave them all as nans.
    """
    if x.isnull().all():
        return np.nan
    else:
        mode = x.mode()[0]
        return x.fillna(mode)

    
def fill_nans(df):
    """ Fill nan spaces with the mode of the group by ref_hash. """
    nans_filled = df
    nans_filled = nans_filled.sort_values(by=['ref_hash','date'])

    asdf = nans_filled.groupby('ref_hash', as_index=False, sort=False).transform(lambda x: x.ffill().bfill())

    asdf['ref_hash'] = nans_filled['ref_hash']
    
    return asdf

#     nans_filled = df.groupby('ref_hash', as_index=False, sort=False).transform(lambda x: x.ffill().bfill())
    
#     for col in nans_filled.columns:
#         print("filling column " + col)
# #         col_mode = nans_filled[col].mode()[0]
#         nans_filled[col].fillna(col_mode, inplace=True)
    
#     nans_filled['ref_hash'] = df['ref_hash']
#     return nans_filled

# def fill_all_nans_but_diff_in_sec(df):
#     nans_filled = fill_all_nans(df.drop(['diff_in_sec'],axis='columns'))
#     nans_filled['diff_in_sec'] = df['diff_in_sec']
#     return nans_filled

# def object_to_categorical(df):
#     """
#     Transform all 'object' dtypes to 'category'.
#     The following answers helped address the issue:
#         - https://stackoverflow.com/a/46762926
#         - https://stackoverflow.com/a/39092877
#     """
#     asdf = df
    
#     for col in asdf.columns:
#         if asdf[col].dtype.kind == 'O':
#             print(col)
#             asdf[col] = asdf[col].astype('category')
        
#     return asdf

def col_to_bool(df, cols):
    asdf = df
    for col in cols:
        asdf[col] = asdf[col].astype('bool')
    return df

def get_time_to_event_of_interest(df, source_of_interest):
    asdf = df
    
    asdf = asdf.sort_values(['ref_hash','date'])

    asdf['timestamp_of_next_occurrence'] = asdf['date']
    asdf.loc[asdf['source_csv'] != source_of_interest, 'timestamp_of_next_occurrence'] = np.nan

    # asdf = nans_filled.groupby('ref_hash', as_index=False, sort=False).transform(lambda x: x.ffill().bfill())

    asdf['timestamp_of_next_occurrence'] = asdf['timestamp_of_next_occurrence'].bfill().ffill()

    asdf['diff_in_sec'] = asdf['date']-asdf['timestamp_of_next_occurrence']
    
    asdf['diff_in_sec'] = asdf['diff_in_sec'].dt.total_seconds()

    asdf['diff_in_sec'] = asdf['diff_in_sec'].apply(lambda x: x if x <= secs_in_3_days else secs_in_3_days)
    asdf['diff_in_sec'] = asdf['diff_in_sec'].apply(lambda x: x if x > 0 else secs_in_3_days)
    
    asdf = asdf.drop('timestamp_of_next_occurrence', axis='columns')
    
    return asdf

## Dfs loading functions

Luego de appendear los 4 dfs y rellenar los nans con la moda por cada grupo, se ha observado, con una muestra de device ids (ref_hashes), la siguiente proporcion de nans:

> source_id has 0.0% of nans.<br>
date has 0.0% of nans.<br>
latitude has 68.65% of nans.<br>
longitude has 68.65% of nans.<br>
wifi_connection has 68.65% of nans.<br>
carrier_id has 68.65% of nans.<br>
os_minor has 68.65% of nans.<br>
os_major has 68.65% of nans.<br>
specs_brand has 68.65% of nans.<br>
timeToClick has 68.65% of nans.<br>
touchX has 68.65% of nans.<br>
touchY has 68.65% of nans.<br>
ref_type has 68.65% of nans.<br>
diff_in_sec has 0.0% of nans.<br>
source_csv has 0.0% of nans.<br>
application_id has 0.0% of nans.<br>
attributed has 0.0% of nans.<br>
implicit has 0.0% of nans.<br>
device_brand has 34.18% of nans.<br>
device_model has 2.14% of nans.<br>
session_user_agent has 0.13% of nans.<br>
device_language has 3.31% of nans.<br>
ip_address has 16.25% of nans.<br>
ref_type_id has 0.0% of nans.<br>
ref_hash has 0.0% of nans.<br>

Se decide no trabajar con las columnas que tengan mas de 50% de nans.

Codigo ejecutado:
```python
for col in nans_filled.columns:
    total_rows = nans_filled.shape[0]
    print(str(col) + " has " + str(100*nans_filled[col].isna().sum()/total_rows) + "% of nans.")
```

### Clicks

In [11]:
clicks_cols = ['source_id','created','ref_hash',]

clicks_dtypes = {
#     'advertiser_id':'category',
#                  'action_id':'category',
                 'source_id':'category',
#                  'country_code':'category',
#                  'latitude':'float64',
#                  'longitude':'float64',
#                  'wifi_connection':'bool',
#                  'carrier_id':'category',
#                  'trans_id':'category',
#                  'os_minor':'category',
#                  'agent_device':'category',
#                  'os_major':'category',
#                  'specs_brand':'category',
#                  'brand':'category',
#                  'timeToClick':'float64',
#                  'touchX':'object',
#                  'touchY':'object',
#                  'ref_type':'category',
                 'ref_hash':'category'}

def load_clicks(users=get_target_ids(), days=all_days):
    """
    load clicks csv, only users and days specified in users and days lists.
    If lists left empty, consider whole set of users and days respectively.
    """
    
    df_clicks = pd.read_csv('../data/clicks.csv', engine='c', dtype=clicks_dtypes, parse_dates=['created'], usecols=clicks_cols)
    
    def load_condition(chunk):
        return chunk['ref_hash'].isin(users) & chunk['created'].dt.day.isin(days)
    
    df = df_clicks.loc[load_condition(df_clicks)].copy()
    
    df.rename(columns={'created':'date'}, inplace=True)
    
    df['date'] = df['date'].dt.tz_localize(None)
    
    return df

### Installs

In [12]:
installs_cols = ['created','application_id','ref_hash','attributed','implicit','device_brand','device_model','session_user_agent','device_language']

installs_dtypes = {"application_id":          "category",
#                    "ref_type":                "category",
                   "ref_hash":                "object",
#                    "click_hash":             "category",
                   "attributed":               "bool",
                   "implicit":                 "bool",
#                    "device_countrycode":      "category",
                   "device_brand":          "category",
                   "device_model":          "category",
                   "session_user_agent":     "category",
#                    "user_agent":             "category",
#                    "event_uuid":             "category",
#                    "kind":                   "category",
#                    "wifi":                   "category",
#                    "trans_id":               "category",
#                    "ip_address":              "category",
                   "device_language":       "category"}

def load_installs(users=get_target_ids(), days=all_days):
    """
    load installs csv, only users and days specified in users and days lists.
    If lists left empty, consider whole set of users and days respectively.
    """
    
    df_installs = pd.read_csv('../data/installs.csv', engine='c', dtype=installs_dtypes, parse_dates=['created'], usecols=installs_cols)
    
    def load_condition(df):
        return df['ref_hash'].isin(users) & df['created'].dt.day.isin(days)
    
    df = df_installs.loc[load_condition(df_installs)].copy()
    
    df.rename(columns={'created':'date'}, inplace=True)
    
    return df

### Events

In [13]:
events_cols = ['date','ref_hash','application_id','attributed','device_model','ip_address']

events_dtypes = {
#     "index":                   "category",
#                  "event_id":                "category",
#                  "ref_type":                "category",
                 "ref_hash":                "category",
                 "application_id":          "category",
                 "attributed":               "bool",
#                  "device_countrycode":      "category",
#                  "device_os_version":     "category",
#                  "device_brand":          "category",
                 "device_model":          "category",
#                  "device_city":           "category",
#                  "session_user_agent":    "category",
#                  "trans_id":               "category",
#                  "user_agent":            "category",
#                  "event_uuid":             "category",
#                  "carrier":               "category",
#                  "kind":                  "category",
#                  "device_os":             "category",
#                  "wifi":                     "bool",
#                  "connection_type":        "category",
                 "ip_address":              "category",
#                  "device_language":       "category"
}

def load_events(users=get_target_ids(), days=all_days):
    """
    load events csv, only users and days specified in users and days lists.
    If lists left empty, consider whole set of users and days respectively.
    """
    
    iter_events = pd.read_csv('../data/events.csv', engine='c', dtype=events_dtypes, parse_dates=['date'], chunksize=10000, usecols=events_cols)
    
    def load_condition(chunk):
        return chunk['ref_hash'].isin(users) & chunk['date'].dt.day.isin(days)
    
    df = pd.concat(chunk.loc[load_condition(chunk)] for chunk in iter_events)
    
    return df

### Auctions

In [14]:
auction_cols = ['date','device_id']

auctions_dtypes = {'device_id':'category',
#                  'ref_type_id':'category',
#                  'source_id':'category'
                  }

def load_auctions(users=get_target_ids(), days=all_days):
    """
    load auctions csv, only users and days specified in users and days lists.
    If lists left empty, consider whole set of users and days respectively.
    """
    iter_auctions = pd.read_csv('../data/auctions.csv', engine='c', dtype=auctions_dtypes, parse_dates=['date'], chunksize=10000, usecols=auction_cols)
    def load_condition(chunk):
        return chunk['device_id'].isin(users) & chunk['date'].dt.day.isin(days)
    df = pd.concat(chunk.loc[load_condition(chunk)] for chunk in iter_auctions)
    
    df.rename(columns={'device_id':'ref_hash'}, inplace=True)
    
    return df

# Recommended bibliography

#### Pseudo resumen de cosas utiles para aplicar
Para survival analysis se necesitan dos cosas:
- an array of durations
- either a boolean or binary array representing whether the “death” was observed or not (alternatively an individual can be censored).

# ML: Approaches

***

## Approach 6

In [15]:
current_users = get_target_ids()
current_days = get_n_3_days(1)

In [22]:
df_clicks = load_clicks(current_users, current_days)

In [24]:
df_events = load_events(current_users, current_days)

### Auctions

### 1

In [18]:
df_auctions = load_auctions(current_users, current_days)

In [19]:
df_auctions2 = process_time_diffs(df_auctions)

In [20]:
df_auctions2 = df_auctions2.drop('date', 1)

In [21]:
current_predictions = df_auctions2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [22]:
current_predictions.to_csv('df_auctions_window_1.csv', index = None, header=True)

### 2

In [23]:
current_days = get_n_3_days(2)

In [24]:
df_auctions = load_auctions(current_users, current_days)

In [25]:
df_auctions2 = process_time_diffs(df_auctions)

In [26]:
df_auctions2 = df_auctions2.drop('date', 1)

In [27]:
current_predictions = df_auctions2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [28]:
current_predictions.to_csv('df_auctions_window_2.csv', index = None, header=True)

### 3

In [29]:
current_days = get_n_3_days(3)

In [30]:
df_auctions = load_auctions(current_users, current_days)

In [31]:
df_auctions2 = process_time_diffs(df_auctions)

In [32]:
df_auctions2 = df_auctions2.drop('date', 1)

In [33]:
current_predictions = df_auctions2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [34]:
current_predictions.to_csv('df_auctions_window_3.csv', index = None, header=True)

### 4

In [35]:
current_days = get_n_3_days(4)

In [36]:
df_auctions = load_auctions(current_users, current_days)

In [37]:
df_auctions2 = process_time_diffs(df_auctions)

In [38]:
df_auctions2 = df_auctions2.drop('date', 1)

In [39]:
current_predictions = df_auctions2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [40]:
current_predictions.to_csv('df_auctions_window_4.csv', index = None, header=True)

### 5

In [41]:
current_days = get_n_3_days(5)

In [42]:
df_auctions = load_auctions(current_users, current_days)

In [43]:
df_auctions2 = process_time_diffs(df_auctions)

In [44]:
df_auctions2 = df_auctions2.drop('date', 1)

In [45]:
current_predictions = df_auctions2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [46]:
current_predictions.to_csv('df_auctions_window_5.csv', index = None, header=True)

### 6

In [47]:
current_days = get_n_3_days(6)

In [48]:
df_auctions = load_auctions(current_users, current_days)

In [49]:
df_auctions2 = process_time_diffs(df_auctions)

In [50]:
df_auctions2 = df_auctions2.drop('date', 1)

In [51]:
current_predictions = df_auctions2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [52]:
current_predictions.to_csv('df_auctions_window_6.csv', index = None, header=True)

In [53]:
del df_auctions2
del df_auctions
del current_predictions

### Load csvs

In [15]:
auction_window_1 = pd.read_csv('df_auctions_window_1.csv')
auction_window_2 = pd.read_csv('df_auctions_window_2.csv')
auction_window_3 = pd.read_csv('df_auctions_window_3.csv')
auction_window_4 = pd.read_csv('df_auctions_window_4.csv')
auction_window_5 = pd.read_csv('df_auctions_window_5.csv')
auction_window_6 = pd.read_csv('df_auctions_window_6.csv')

### XGBoost

### Window 1 and 2

In [16]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [17]:
X_train, y_train = auction_window_1.iloc[:,:-1],auction_window_1.iloc[:,-1]

In [18]:
X_test,y_test = auction_window_2.iloc[:,:-1],auction_window_2.iloc[:,-1]

In [19]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [20]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [21]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [22]:
preds = xg_reg.predict(X_test)

In [23]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 21976.192765


### Window 2 and 3

In [24]:
X_train, y_train = auction_window_2.iloc[:,:-1],auction_window_2.iloc[:,-1]

In [25]:
X_test,y_test = auction_window_3.iloc[:,:-1],auction_window_3.iloc[:,-1]

In [26]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [27]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [28]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [29]:
preds = xg_reg.predict(X_test)

In [30]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 22293.294801


### Window 3 and 4

In [31]:
X_train, y_train = auction_window_3.iloc[:,:-1],auction_window_3.iloc[:,-1]

In [32]:
X_test,y_test = auction_window_4.iloc[:,:-1],auction_window_4.iloc[:,-1]

In [33]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [34]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [35]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [36]:
preds = xg_reg.predict(X_test)

In [37]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 22568.301623


### Window 4 and 5

In [38]:
X_train, y_train = auction_window_4.iloc[:,:-1],auction_window_4.iloc[:,-1]

In [39]:
X_test,y_test = auction_window_5.iloc[:,:-1],auction_window_5.iloc[:,-1]

In [40]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [41]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [42]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [43]:
preds = xg_reg.predict(X_test)

In [44]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 23252.154421


### Window 5 and 6

In [45]:
X_train, y_train = auction_window_5.iloc[:,:-1],auction_window_5.iloc[:,-1]

In [46]:
X_test,y_test = auction_window_6.iloc[:,:-1],auction_window_6.iloc[:,-1]

In [47]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [48]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [49]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [50]:
preds = xg_reg.predict(X_test)

In [51]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 22873.055770


### Load target

In [52]:
targets_st = target[target['ref_hash'].str.endswith('_st')]

In [53]:
targets_st['ref_hash'] = targets_st['ref_hash'].str[:-3]

/home/fabrizio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
targets_st['ref_hash'] = targets_st['ref_hash'].astype(np.int64)

/home/fabrizio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
# Carga de archivos CSV
dfAuctions = pd.read_csv('../data/auctions.csv', 
                 engine='c', 
                 usecols=['device_id','ref_type_id', 'source_id'], 
                 dtype={'ref_type_id':np.int8, 'source_id':np.int8})

In [56]:
dfAuctions = dfAuctions.drop_duplicates(subset = 'device_id')

In [57]:
targets_st.columns = ['device_id','obj']

In [58]:
targets_st = targets_st.merge(dfAuctions[['device_id']], on = 'device_id', how = 'left')

In [59]:
targets_st.fillna(0, inplace=True)

In [61]:
targets_st = targets_st.drop(columns = ['obj'])

In [62]:
targets_st.columns = ['ref_hash']

In [63]:
X_train, y_train = auction_window_6.iloc[:,:-1],auction_window_6.iloc[:,-1]

In [64]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [65]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [66]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [67]:
preds_st = xg_reg.predict(targets_st)

In [68]:
target_preds_st = target[target['ref_hash'].str.contains('_st')]
target_preds_st['obj'] = preds_st

/home/fabrizio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [49]:
del df_auctions
del df_auctions2

### Installs

In [114]:
current_users = get_target_ids()
current_days = get_n_3_days(1)

### 1

In [115]:
df_installs = load_installs(current_users, current_days)

In [116]:
df_installs2 = process_time_diffs(df_installs)

In [117]:
df_installs2 = df_installs2.drop('date', 1)

In [118]:
current_predictions = df_installs2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [119]:
current_predictions.to_csv('df_installs_window_1.csv', index = None, header=True)

### 2

In [120]:
current_days = get_n_3_days(2)

In [121]:
df_installs = load_installs(current_users, current_days)

In [122]:
df_installs2 = process_time_diffs(df_installs)

In [123]:
df_installs2 = df_installs2.drop('date', 1)

In [124]:
current_predictions = df_installs2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [125]:
current_predictions.to_csv('df_installs_window_2.csv', index = None, header=True)

### 3

In [126]:
current_days = get_n_3_days(3)

In [127]:
df_installs = load_installs(current_users, current_days)

In [128]:
df_installs2 = process_time_diffs(df_installs)

In [129]:
df_installs2 = df_installs2.drop('date', 1)

In [130]:
current_predictions = df_installs2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [131]:
current_predictions.to_csv('df_installs_window_3.csv', index = None, header=True)

### 4

In [132]:
current_days = get_n_3_days(4)

In [133]:
df_installs = load_installs(current_users, current_days)

In [134]:
df_installs2 = process_time_diffs(df_installs)

In [135]:
df_installs2 = df_installs2.drop('date', 1)

In [136]:
current_predictions = df_installs2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [137]:
current_predictions.to_csv('df_installs_window_4.csv', index = None, header=True)

### 5

In [138]:
current_days = get_n_3_days(5)

In [139]:
df_installs = load_installs(current_users, current_days)

In [140]:
df_installs2 = process_time_diffs(df_installs)

In [141]:
df_installs2 = df_installs2.drop('date', 1)

In [142]:
current_predictions = df_installs2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [143]:
current_predictions.to_csv('df_installs_window_5.csv', index = None, header=True)

### 6

In [144]:
current_days = get_n_3_days(6)

In [145]:
df_installs = load_installs(current_users, current_days)

In [146]:
df_installs2 = process_time_diffs(df_installs)

In [147]:
df_installs2 = df_installs2.drop('date', 1)

In [148]:
current_predictions = df_installs2.groupby('ref_hash', as_index=False)[['diff_in_sec']].mean()

In [149]:
current_predictions.to_csv('df_installs_window_6.csv', index = None, header=True)

In [150]:
del df_installs2
del df_installs
del current_predictions

### Load csvs

In [82]:
installs_window_1 = pd.read_csv('df_installs_window_1.csv')
installs_window_2 = pd.read_csv('df_installs_window_2.csv')
installs_window_3 = pd.read_csv('df_installs_window_3.csv')
installs_window_4 = pd.read_csv('df_installs_window_4.csv')
installs_window_5 = pd.read_csv('df_installs_window_5.csv')
installs_window_6 = pd.read_csv('df_installs_window_6.csv')

### XGBoost

### Window 1 and 2

In [83]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [84]:
X_train, y_train = installs_window_1.iloc[:,:-1],installs_window_1.iloc[:,-1]

In [85]:
X_test,y_test = installs_window_2.iloc[:,:-1],installs_window_2.iloc[:,-1]

In [86]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [87]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [88]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [89]:
preds = xg_reg.predict(X_test)

In [90]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 29529.687908


### Window 2 and 3

In [91]:
X_train, y_train = installs_window_2.iloc[:,:-1],installs_window_2.iloc[:,-1]

In [92]:
X_test,y_test = installs_window_3.iloc[:,:-1],installs_window_3.iloc[:,-1]

In [93]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [94]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [95]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [96]:
preds = xg_reg.predict(X_test)

In [97]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 29967.764219


### Window 3 and 4

In [98]:
X_train, y_train = installs_window_3.iloc[:,:-1],installs_window_3.iloc[:,-1]

In [99]:
X_test,y_test = installs_window_4.iloc[:,:-1],installs_window_4.iloc[:,-1]

In [100]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [101]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [102]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [103]:
preds = xg_reg.predict(X_test)

In [104]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 29974.186441


### Window 4 and 5

In [105]:
X_train, y_train = installs_window_4.iloc[:,:-1],installs_window_4.iloc[:,-1]

In [106]:
X_test,y_test = installs_window_5.iloc[:,:-1],installs_window_5.iloc[:,-1]

In [107]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [108]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [109]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [110]:
preds = xg_reg.predict(X_test)

In [111]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 30747.226603


### Window 5 and 6

In [112]:
X_train, y_train = installs_window_5.iloc[:,:-1],installs_window_5.iloc[:,-1]

In [113]:
X_test,y_test = installs_window_6.iloc[:,:-1],installs_window_6.iloc[:,-1]

In [114]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [115]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [116]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [117]:
preds = xg_reg.predict(X_test)

In [118]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 30337.516312


### Load target

In [119]:
targets_sc = target[target['ref_hash'].str.endswith('_sc')]

In [120]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].str[:-3]

/home/fabrizio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
targets_sc['ref_hash'] = targets_sc['ref_hash'].astype(np.int64)

/home/fabrizio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [122]:
# Carga de archivos CSV
dfInstalls = pd.read_csv('../data/installs.csv',
                 engine='c', 
                 usecols=['ref_hash'])

In [123]:
dfInstalls = dfInstalls.drop_duplicates(subset = 'ref_hash')

In [124]:
targets_sc = targets_sc.merge(dfInstalls[['ref_hash']], on = 'ref_hash', how = 'left')

In [125]:
targets_sc.fillna(0, inplace=True)

In [126]:
targets_sc = targets_sc.drop(columns = ['obj'])

In [127]:
X_train, y_train = installs_window_6.iloc[:,:-1],installs_window_6.iloc[:,-1]

In [128]:
# solo datos numericos
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)

In [129]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 6, alpha = 10, n_estimators = 12,
                missing=-999, seed=1337)

In [130]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=-999, n_estimators=12,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1337,
       silent=True, subsample=1)

In [131]:
preds_sc = xg_reg.predict(targets_sc)

In [132]:
target_preds_sc = target[target['ref_hash'].str.contains('_sc')]
target_preds_sc['obj'] = preds_sc

/home/fabrizio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [134]:
target_preds_sc.head()

,ref_hash,obj
0,1000169251625791246_sc,32630.986328
2,1000395625957344683_sc,32630.986328
4,1003027494996471685_sc,32630.986328
6,1006670001679961544_sc,32630.986328
8,1007573308966476713_sc,32630.986328


In [133]:
target_preds_st.head()

,ref_hash,obj
1,1000169251625791246_st,14093.609375
3,1000395625957344683_st,14093.609375
5,1003027494996471685_st,14093.609375
7,1006670001679961544_st,14093.609375
9,1007573308966476713_st,14093.609375


In [136]:
target_to_save = [target_preds_sc, target_preds_st]

In [137]:
target_to_save = pd.concat(target_to_save)

In [140]:
save_submission(target_to_save, description="por cada grupo, avg. de los tiempos entre cada registro con xgboost.")

In [51]:
# df_installs = load_installs()

# df_installs2 = process_time_diffs(df_installs)

# current_predictions = df_installs2[['ref_hash','diff_in_sec']].groupby('ref_hash', as_index=False).mean()

# target = store_predictions(target_df=target, new_values=current_predictions, value_column_name='diff_in_sec', suffix='_sc')

In [50]:
del df_installs
del df_installs2

***

In [57]:
# save_submission(target, description="por cada grupo, avg. de los tiempos entre cada registro.")

## End of Approach 6